# ベースモデル

## 5/11 1500: validation loss = 0.31292   validation accuracy = 0.86842
StratifiedGraoupKfoldで安定的なCVの作成

fold 3, epoch 1~5




In [1]:
#ColaboratoryかKaggleNotebookか判別
import sys
import os
from pathlib import Path

WORKING_DIR = '/content/drive/MyDrive/git/kaggle-inclass-brest-cancer-classification/working'  # コンペごとに書き換える
MAIN_INPUT_NAME = "brest-cancer-classification"     # コンペごとに書き換える

INPUT = Path('../input/')
MAIN_INPUT_ZIP = f'../input/{MAIN_INPUT_NAME}.zip'
if 'google.colab' in sys.modules:  # colab環境
    INPUT = Path('/content/input/')

    # drive mount
    from google.colab import drive
    drive.mount('/content/drive/')

    # working dir に移動
    os.chdir(WORKING_DIR)
    print("cwd:", os.getcwd())

    if not os.path.exists("/content/input"):
        # 必要な input を指定先へ unzip
        !unzip -q {MAIN_INPUT_ZIP} -d {INPUT}
        print("unzip INPUT")
    
elif 'kaggle_web_client' in sys.modules:  # kaggle環境
    pass

!pip install timm
!pip install albumentations==0.4.5
!pip install --pre --extra-index https://pypi.anaconda.org/scipy-wheels-nightly/simple scikit-learn==1.0.dev0

Mounted at /content/drive/
cwd: /content/drive/MyDrive/git/kaggle-inclass-brest-cancer-classification/working
unzip INPUT
     |████████████████████████████████| 296kB 5.3MB/s 
     |████████████████████████████████| 122kB 4.9MB/s 
     |████████████████████████████████| 634kB 8.5MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.5-cp37-none-any.whl size=64378 sha256=b6ec79e2b4ccfc40660a291f18f838b668b6e832b333810d9306e01ed240ece8
  Stored in directory: /root/.cache/pip/wheels/f0/a0/61/e50f93165a5ec7e7f5d65064e513239505bc4c06d2289557d3
  Created wheel for imgaug: filename=imgaug-0.2.6-cp37-none-any.whl size=654019 sha256=7d6fccae377ca46340768c735b074cc2bf01b87cb64204d39181af317b19c920
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built albumentations imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing ins

In [2]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## ライブラリ

In [3]:
import json
import os
import random
import time

import numpy as np
import pandas as pd
import cv2

import torch
from torch import nn
from torch.utils.data import Dataset
from torch.cuda.amp import autocast, GradScaler

import timm
from tqdm.notebook import tqdm

from sklearn.model_selection import StratifiedKFold, GroupKFold, StratifiedGroupKFold
from sklearn.metrics import log_loss, f1_score

In [ ]:
# train load

def load_train_df():
    base_path = os.path.join(INPUT, "train_image")
    df = pd.read_csv(os.path.join(INPUT, "train.csv"), index_col=0)
    df = pd.concat(
        [
          df,
          df['img_name'].str.rstrip(".png").str.split('_', expand=True).rename(columns={0: 'exam', 1: 'x', 2: 'y'}, inplace=False)
        ],
        axis=1
    )
    df["exam"] = df["exam"].astype(int)
    df["x"] = df["x"].str.lstrip("x_").astype(int)
    df["y"] = df["y"].str.lstrip("y_").astype(int)
    df["img_path"] = base_path + "/" + df["label"].astype(str).str.cat(df['img_name'], sep=os.sep)
    return df
train_df = load_train_df()
print(train_df.shape)
train_df.head()

(175889, 6)


,img_name,label,exam,x,y,img_path
0,8863_x2301_y1751.png,0,8863,2301,1751,/content/input/train_image/0/8863_x2301_y1751.png
1,8863_x1601_y751.png,0,8863,1601,751,/content/input/train_image/0/8863_x1601_y751.png
2,8863_x901_y1401.png,0,8863,901,1401,/content/input/train_image/0/8863_x901_y1401.png
3,8863_x651_y1101.png,0,8863,651,1101,/content/input/train_image/0/8863_x651_y1101.png
4,8863_x351_y651.png,0,8863,351,651,/content/input/train_image/0/8863_x351_y651.png


In [ ]:
def load_test_df():
    base_path = os.path.join(INPUT, "test_image/test_image")
    df = pd.read_csv(os.path.join(INPUT, "submission.csv"))
    df = pd.concat(
        [
          df,
          df['Id'].str.rstrip(".png").str.split('_', expand=True).rename(columns={0: 'exam', 1: 'x', 2: 'y'}, inplace=False)
        ],
        axis=1
    )
    df["exam"] = df["exam"].astype(int)
    df["x"] = df["x"].str.lstrip("x_").astype(int)
    df["y"] = df["y"].str.lstrip("y_").astype(int)
    df["img_path"] = base_path + "/" +  df['Id']
    return df

test_df = load_test_df()
print(test_df.shape)
test_df.head()

(101635, 6)


,Id,label,exam,x,y,img_path
0,9381_x951_y601.png,0,9381,951,601,/content/input/test_image/test_image/9381_x951...
1,9381_x951_y2651.png,0,9381,951,2651,/content/input/test_image/test_image/9381_x951...
2,9381_x951_y2001.png,0,9381,951,2001,/content/input/test_image/test_image/9381_x951...
3,9381_x951_y1951.png,0,9381,951,1951,/content/input/test_image/test_image/9381_x951...
4,9381_x951_y1901.png,0,9381,951,1901,/content/input/test_image/test_image/9381_x951...
